# Setup

Initial module setup.

In [ ]:
import logging
import typing
import time
import statistics
import numpy.typing as np_types
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from eeg_auth_models_framework import data, pre_process, features, training, model, processor
from eeg_auth_models_framework.utils import conversion

# Constants

In [21]:
DATASET_SAMPLE_FREQ_HZ = 200
DATA_CHANNEL_NAMES = ['T7','F8','Cz','P4']
FREQUENCIES = [
    pre_process.FrequencyBand(lower=8.0, upper=12.0, label='Alpha'),
    pre_process.FrequencyBand(lower=12.0, upper=35.0, label='Beta'),
    pre_process.FrequencyBand(lower=4.0, upper=8.0, label='Theta'),
    pre_process.FrequencyBand(lower=35.0, upper=None, label='Gamma'),
    pre_process.FrequencyBand(lower=None, upper=None, label='Raw'),
]
WINDOW_SIZE = 1200
WINDOW_OVERLAP = 0.5
K_FOLDS = 10
RANDOM_SEED = 42

# Model Builder Configuration

Configure data source, data reading method, data labelling method, and training process. 

In [22]:
class MusicIDModelBuilder(model.ModelBuilder[RandomForestClassifier]):
    def create_classifier(self) -> RandomForestClassifier:
        return RandomForestClassifier(
            n_estimators=100,
            criterion='gini',
            max_depth=10,
            random_state=RANDOM_SEED
        )
    
    def train_classifier(self, classifier: RandomForestClassifier, x_data: np_types.ArrayLike, y_data: np_types.ArrayLike):
        classifier.fit(x_data, y_data)
        
    def score_classifier(self, classifier: RandomForestClassifier, x_data: np_types.ArrayLike, y_data: np_types.ArrayLike) -> float:
        return classifier.score(x_data, y_data)

# Configuration

In [23]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
downloader = data.AuditoryDataDownloader()
reader = data.AuditoryDataReader()
labeller = training.SubjectDataLabeller()
converter = conversion.MNEDataFrameConverter(
    channels=DATA_CHANNEL_NAMES, 
    sample_frequency=DATASET_SAMPLE_FREQ_HZ
)

# Data Processing

## Pre-Processing Steps

Define pre-processing steps to be used in model.

In [24]:
pre_process_steps = pre_process.PreProcessingPipeline([
    pre_process.EEGBandpassFilterStep(
        FREQUENCIES,
        converter
    ),
    pre_process.DataWindowStep(WINDOW_SIZE, WINDOW_OVERLAP)
])

## Feature Extraction Steps

Define feature extraction steps to be applied to the pre-processed data.

In [25]:
feature_extraction_steps = features.FeatureExtractPipeline([
    features.StatisticalFeatureExtractor([
        features.StatisticalFeature.MIN,
        features.StatisticalFeature.MAX,
        features.StatisticalFeature.MEAN,
        features.StatisticalFeature.ZERO_CROSSING_RATE
    ])
])

## Data Processor

In [26]:
data_processor = processor.DataProcessor(
    pre_process=pre_process_steps,
    feature_extraction=feature_extraction_steps
)

# Training

Execute training of authentication models.

In [27]:
music_id_builder = MusicIDModelBuilder(
    data_downloader=downloader,
    data_reader=reader,
    data_labeller=labeller,
    data_processor=data_processor
)
results = music_id_builder.train(K_FOLDS)

Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=40114
    Range : 0 ... 40113 =      0.000 ...   200.565 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ..

In [28]:
subject_score_rows = []
for subject in results.training_statistics:
    subject_score_rows.append([
        subject,
        results.training_statistics[subject].average_score,
        results.training_statistics[subject].training_duration
    ])
subject_scores = pd.DataFrame(subject_score_rows, columns=['Subject', 'Average Score', 'Training Duration'])
subject_scores

,Subject,Average Score,Training Duration
0,S01,0.951852,1.052002
1,S02,0.951852,1.241000
2,S03,0.950617,1.093000
3,S04,1.000000,0.939000
4,S05,0.998765,1.172999
5,S06,0.998750,1.019500
6,S07,1.000000,0.968500
7,S08,1.000000,1.086002
8,S09,1.000000,1.034502
9,S10,1.000000,1.031000


In [29]:
subject_rates_rows = []
for subject in results.training_statistics:
    subject_rates_rows.append([
        subject,
        results.training_statistics[subject].average_false_accept_rate,
        results.training_statistics[subject].average_false_reject_rate
    ])
subject_rates = pd.DataFrame(subject_rates_rows, columns=['Subject', 'Average FAR', 'Average FRR'])
subject_rates

,Subject,Average FAR,Average FRR
0,S01,0.050649,0.000000
1,S02,0.050649,0.000000
2,S03,0.051948,0.000000
3,S04,0.000000,0.000000
4,S05,0.000000,0.016667
5,S06,0.001316,0.000000
6,S07,0.000000,0.000000
7,S08,0.000000,0.000000
8,S09,0.000000,0.000000
9,S10,0.000000,0.000000


In [30]:
global_stats = pd.DataFrame(
    [[results.global_average_score, results.global_average_far_rate, 
      results.global_average_frr_rate, results.global_average_time]],
    columns=['Global Average Score', 'Global Average FAR', 'Global Average FRR', 'Global Average Time']
)
global_stats

,Global Average Score,Global Average FAR,Global Average FRR,Global Average Time
0,0.989165,0.009583,0.010875,1.082851


# Simulated Execution

In [ ]:
def simulate_model_executions(models: typing.Dict[str, RandomForestClassifier], 
                              test_downloader: data.AuditoryDataDownloader, 
                              test_reader: data.AuditoryDataReader,
                              test_processor: processor.DataProcessor,
                              tests = 10) -> typing.Tuple[int, float]:
    data_path = test_downloader.retrieve()
    test_data = test_reader.format_data(data_path)
    for subject, data_samples in test_data.items():
        test_data[subject] = test_processor.process(data_samples)
    predict_timings = []
    for i in range(tests):
        for subject, prepared_data in test_data.items():
            test_model = models[subject]
            start_predict = time.perf_counter()
            test_model.predict(prepared_data, fhe="execute")
            end_predict = time.perf_counter()
            predict_timings.append(end_predict - start_predict)
    return len(predict_timings), statistics.mean(predict_timings)

In [ ]:
executions, average_predict_time = simulate_model_executions(results.models, downloader, reader, data_processor)
print(f"Executions: {executions}")
print(f"Overall average predict time: {average_predict_time} seconds")